List users and roles they are mapped to

In [ ]:
DECLARE @DB_USers TABLE
(DBName sysname, UserName sysname, LoginType sysname, AssociatedRole varchar(max),create_date datetime,modify_date datetime)

INSERT @DB_USers
EXEC sp_MSforeachdb

'
use [?]
SELECT ''?'' AS DB_Name,
case prin.name when ''dbo'' then prin.name + '' (''+ (select SUSER_SNAME(owner_sid) from master.sys.databases where name =''?'') + '')'' else prin.name end AS UserName,
prin.type_desc AS LoginType,
isnull(USER_NAME(mem.role_principal_id),'''') AS AssociatedRole ,create_date,modify_date
FROM sys.database_principals prin
LEFT OUTER JOIN sys.database_role_members mem ON prin.principal_id=mem.member_principal_id
WHERE prin.sid IS NOT NULL and prin.sid NOT IN (0x00) and
prin.is_fixed_role <> 1 AND prin.name NOT LIKE ''##%'''

SELECT

dbname,username ,logintype ,create_date ,modify_date ,AssociatedRole

FROM @DB_USers 


Row-Level security example

In [ ]:
--*******************
--Row Level Security
--*******************
--Show no security
SELECT *
FROM [DataWarehouse].[DimCountry_SecurityDemo]

SELECT *
FROM DataWarehouse.FactAnnualBudget

SELECT  *
FROM DataWarehouse.FactVaR
WHERE COBDateId = 20170201

--Apply Security
CREATE FUNCTION dbo.LimitCountryAccess(@CountryId Int)
RETURNS TABLE
WITH SCHEMABINDING
AS
  RETURN 
  (
    SELECT [Go] = 1 FROM [DataWarehouse].[DimCountry_SecurityDemo]
      WHERE 
      (
	    [DimCountryId] = @CountryId
        AND IS_MEMBER([Security]) = 1
	) --OR IS_SRVROLEMEMBER(N'sysadmin') = 1
  );
GO

CREATE SECURITY POLICY CountryPolicy
ADD FILTER PREDICATE dbo.LimitCountryAccess([DimCountryId]) ON [DataWarehouse].[DimCountry_SecurityDemo]
,ADD FILTER PREDICATE dbo.LimitCountryAccess([CountryId]) ON [DataWarehouse].[FactAnnualBudget]
,ADD FILTER PREDICATE dbo.LimitCountryAccess([CountryId]) ON [DataWarehouse].[FactVaR]
WITH (STATE = ON);

--Show that security is applied
SELECT *
FROM [DataWarehouse].[DimCountry_SecurityDemo]

SELECT *
FROM DataWarehouse.FactAnnualBudget

SELECT  *
FROM DataWarehouse.FactVaR
WHERE COBDateId = 20170201

--Cleanup
DROP SECURITY POLICY [SCMBNT1\CIBR_CR_Report Group].[CountryPolicy]
DROP FUNCTION dbo.LimitCountryAccess

--************
--Data Masking
--************
--Add Mask
ALTER TABLE [DataWarehouse].[DimCountry_SecurityDemo]
ALTER COLUMN Country ADD MASKED WITH (FUNCTION = 'default()');

--I'm dbo so I can see
SELECT *
FROM [DataWarehouse].[DimCountry_SecurityDemo]

CREATE USER Test WITHOUT LOGIN;
GRANT SELECT ON [DataWarehouse].[DimCountry_SecurityDemo] TO Test;
EXECUTE AS USER = 'Test';
SELECT * FROM [DataWarehouse].[DimCountry_SecurityDemo];
REVERT

GRANT UNMASK TO Test;
EXECUTE AS USER = 'Test';
SELECT * FROM [DataWarehouse].[DimCountry_SecurityDemo];
REVERT


3_~